In [ ]:
import os
import re
import sys
import s3fs
import json
import glob
import git
import xarray
import astral
import rioxarray
import datetime
import rasterio
import tifffile
import subprocess
import numpy as np
import pandas as pd

import pyproj
from pyproj import Proj
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from managers import utils, managers

%load_ext autoreload
%autoreload 2

In [ ]:
a = astral.Astral()
a.solar_depression = 'civil'
city = a['San Francisco']
sun = city.sun(date=datetime.date(2019, 10, 5), local=True)

In [ ]:
sun.get('sunset').astimezone(datetime.timezone.utc).strftime('%j/%H/')

In [ ]:
(sun.get('sunset') - datetime.timedelta(hours=2)).astimezone(datetime.timezone.utc).strftime('%j/%H/')

### Download data

https://docs.opendata.aws/noaa-goes16/cics-readme.html

In [ ]:
import s3fs

# Use the anonymous credentials to access public data
fs = s3fs.S3FileSystem(anon=True)

# List contents of GOES-16 bucket.
fs.ls('s3://noaa-goes16/')


# List specific files of GOES-17 CONUS data (multiband format) on a certain hour
filenames = np.array(fs.ls('s3://noaa-goes17/ABI-L1b-RadC/2019/279/01/'))

rows = []
for filename in filenames:
    result = re.findall('ABI-L1b-RadC-M6C0([1-9])_G17_s([0-9]{14})', filename)
    if result:
        result = result[0]
        rows.append({
            'filename': filename,
            'channel': result[0], 
            'timestamp': datetime.datetime.strptime(result[1][:-1], '%Y%j%H%M%S')
        })

pd.DataFrame(data=rows)
# fs.get(files[0], files[0].split('/')[-1])

In [ ]:
# band 3 (veggie) from daytime 10/5 (DOY 278 at UTC 20:00)
'/home/keith/Downloads/OR_ABI-L1b-RadC-M6C03_G17_s20192782201196_e20192782203569_c20192782204022.nc'

In [ ]:
# 2019-08-31 at 7AM
# (note that band 2 looks cool using plt.imshow(utils.autoscale(im, minn=155, maxx=333, gamma=.7, dtype='uint8'), cmap='gray'))
filepaths = glob.glob('/home/keith/raster-data/GOES-R/2019-08-31_07-01-AM-PST/*.nc')

In [ ]:
# 2019-10-03 at 5PM
filepaths = glob.glob('/home/keith/raster-data/GOES-R/2019-10-03_05-01-PM-PST/*.NC')

In [ ]:
# 2019-10-05 at 6PM
filepaths = glob.glob('/home/keith/raster-data/GOES-R/2019-10-05_06-01-PM-PST/*.nc')

In [ ]:
# clear morning and evening in CA (with mesoscale floater over CA): 2019-08-31
# clear evening in CA: 2019-10-22
# clear evening from seattle to LA: 2019-10-29
# clear morning and evening from Seattle-LA to the rockies (though with snow in the rockies/montana): 2019-11-01

In [ ]:
filepaths

### Creating a GeoTIFF using rio warp

For some reason, this is very slow (1-2min) for band 2.

In [ ]:
# create TIFFs from the netCDF files
for filepath in filepaths:
    utils.run_command([
        'rio', 'warp',
        'NETCDF:%s:Rad' % filepath,
        '%s.tif' % filepath])

### Creating geoTIFFs manually using pyproj and rasterio

Had to figure out how to create GeoTIFFs from xarray-opened NetCDF files because using GDAL is so slow (which seems to be a known issue). First tried using rioxarray, but this doesn't work, because rioxarray does not transform the bounds from radians (as they appear in the netcdf file) to meters. Instead, constructed the projection and image bounds manually and saved the image as a GeoTIFF using rasterio itself. The trickiest part was mapping the parameters in the goes_imager_projection to the pyproj.Proj arguments, and also figuring out to multiply the x/y coordinates by the satellite height to transform from radians to meters. (I determined that all of this was 'correct' by using the transform and CRS of the GDAL-generated GeoTIFFs as a reference.)

Some hints about the projection parameters are here: https://github.com/blaylockbk/pyBKB_v2/blob/master/BB_goes16/mapping_GOES16_data.ipynb

In [ ]:
for filepath in glob.glob('/home/keith/raster-data/GOES-R/test-rioxarray/*.nc'):
    nc = xarray.open_dataset(filepath)
    print(filepath)
    
    # construct the projection
    proj_sweep = nc.goes_imager_projection.sweep_angle_axis
    proj_h = nc.goes_imager_projection.perspective_point_height
    proj_lon_0 = nc.goes_imager_projection.longitude_of_projection_origin
    proj = Proj(proj='geos', h=proj_h, lon_0=proj_lon_0, sweep=proj_sweep, ellps='GRS80')
    
    # construct the bounds
    height, width = nc.Rad.shape
    x_bounds = nc.x_image_bounds.values * proj_h
    y_bounds = nc.y_image_bounds.values * proj_h
    x_res = (x_bounds.max() - x_bounds.min())/width
    y_res = (y_bounds.max() - y_bounds.min())/height
    
    # construct the transform for rasterio
    transform = [x_res, 0, x_bounds.min(), 0, y_res, y_bounds.min()]
    
    with rasterio.open(
        '%s.tif' % filepath, 
        mode='w', 
        driver='GTiff', 
        dtype='int16',
        width=width, 
        height=height,
        crs=proj.srs, 
        transform=transform, 
        tiled=False,
        nodata=1023,
        count=1) as dst:
        
        # note that the array (in nc.Rad) is upside down
        dst.write(nc.Rad.values[::-1, :].astype('int16'), 1)

### Rread the resulting TIFF to check for non-null values

In [ ]:
im = tifffile.imread('%s.tif' % filepaths[0])
im.dtype, im.shape, im.min(), im.max()

In [ ]:
plt.figure(figsize=(24,12))
plt.imshow(utils.autoscale(im[::-2, ::2], minn=0, maxx=100), cmap='gray')

### Create a project manager

In [ ]:
# bounds for central Sierra 
latlon_bounds = [-119.8, 37.2, -117.8, 38.2]

# bounds for west coast to SLC
latlon_bounds = [-125, 33, -105, 45]

south_sierra = [-121, 36, -116, 39]

In [ ]:
p = managers.GOESProject(
    project_root='/home/keith/raster-data/GOES-R/test-rioxarray-project',
    dataset_paths='/home/keith/raster-data/GOES-R/test-rioxarray/',
    reset=True)

In [ ]:
p.raw_datasets[0].extant_bands

In [ ]:
p.warp(p.raw_datasets, res=1000, bounds=latlon_bounds, crs='EPSG:3857')

In [ ]:
imR = tifffile.imread(p.get_operation(-1).destination.filepath(2))
imG = tifffile.imread(p.get_operation(-1).destination.filepath(3))
imB = tifffile.imread(p.get_operation(-1).destination.filepath(1))

imR.dtype, imR.shape, imR.min(), imR.max()

In [ ]:
imG_adj = .45*imR + .1*imG + .45*imB
imG_blend = .7*imG_adj + .3*imG

In [ ]:
# 99th percentile for south sierra ROI for 2019-10-05_06-01-PM-PST
{'r': array([159., 289.]),
 'g': array([35., 99.]),
 'g_adj': array([ 98.65, 167.5 ]),
 'g_blend': array([ 80.525  , 143.58899]),
 'b': array([50., 69.])}

In [ ]:
{
'r': np.percentile(imR, (.1, 99.9)),
'g': np.percentile(imG, (.1, 99.9)),
'g_adj': np.percentile(imG_adj, (.1, 99.9)),
'g_blend': np.percentile(imG_blend, (.1, 99.9)),
'b': np.percentile(imB, (.1, 99.9)),
}

In [ ]:
imRGB = np.concatenate((
        utils.autoscale(imR, minn=159, maxx=289, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imG_adj, minn=102, maxx=169, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imB, minn=55, maxx=77, gamma=.9, dtype='uint8')[:, :, None],
    ), axis=2)

In [ ]:
imRGB = np.concatenate((
        utils.autoscale(imR, minn=163, maxx=283, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imG_adj, minn=102, maxx=169, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imB, minn=55, maxx=77, gamma=.9, dtype='uint8')[:, :, None],
    ), axis=2)

In [ ]:
imRGB = np.concatenate((
        utils.autoscale(imR, percentile=99, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imG_adj, percentile=99, gamma=.7, dtype='uint8')[:, :, None], 
        utils.autoscale(imB, percentile=99, gamma=.9, dtype='uint8')[:, :, None],
    ), axis=2)

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(imRGB)

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(utils.autoscale(im, minn=155, maxx=333, gamma=.7, dtype='uint8'), cmap='gray')

In [ ]:
p.project_root

In [ ]:
tifffile.imwrite(os.path.join(p.project_root, 'tmp_R-Gadj-B.tif'), imRGB)